In [1]:
import cv2
import numpy as np
import pandas as pd
import time 
import math

In [2]:
detector = cv2.dnn.readNetFromCaffe('deploy.prototxt', 'res10_300x300_ssd_iter_140000.caffemodel')

In [3]:
image = cv2.imread("qwe_download.jpg")
base_img = image.copy()

In [4]:
original_size = image.shape
target_size = (300, 300)

In [5]:
image = cv2.resize(image, target_size)

In [6]:
aspect_ratio_x = (original_size[1] / target_size[1])
aspect_ratio_y = (original_size[0] / target_size[0])

In [7]:
image.shape

(300, 300, 3)

In [8]:
cv2.imshow('image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
imageBlob = cv2.dnn.blobFromImage(image = image)

In [10]:
detector.setInput(imageBlob)
detections = detector.forward()

In [11]:
detections[0][0].shape

(200, 7)

In [12]:
detections_df = pd.DataFrame(detections[0][0]
    , columns = ["img_id", "is_face", "confidence", "left", "top", "right", "bottom"])

In [13]:
# change confidence ratio here 

detections_df = detections_df[detections_df['is_face'] == 1] #0: background, 1: face
detections_df = detections_df[detections_df['confidence'] >= 0.50]

In [14]:
detections_df.head()

,img_id,is_face,confidence,left,top,right,bottom
0,0.0,1.0,0.999837,0.351062,0.522484,0.467391,0.823327
1,0.0,1.0,0.988509,0.463291,0.217970,0.581529,0.472572
2,0.0,1.0,0.940376,0.188052,0.500043,0.290668,0.775342
3,0.0,1.0,0.929991,0.018199,0.498472,0.125849,0.765208
4,0.0,1.0,0.869219,0.782433,0.697233,0.898174,0.960454


In [15]:
for i, instance in detections_df.iterrows():
    #print(instance)
    
    confidence_score = str(round(100*instance["confidence"], 2))+" %"
    
    left = int(instance["left"] * 300)
    bottom = int(instance["bottom"] * 300)
    right = int(instance["right"] * 300)
    top = int(instance["top"] * 300)
        
    #low resolution
    #detected_face = image[top:bottom, left:right]
    
    #high resolution
    detected_face = base_img[int(top*aspect_ratio_y):int(bottom*aspect_ratio_y), int(left*aspect_ratio_x):int(right*aspect_ratio_x)]
    
    if detected_face.shape[0] > 0 and detected_face.shape[1] > 0:
        
        #plt.figure(figsize = (3, 3))
        
        #low resolution
        #cv2.putText(image, confidence_score, (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        #cv2.rectangle(image, (left, top), (right, bottom), (255, 255, 255), 1) #draw rectangle to main image
        
        #high resolution
        cv2.putText(base_img, confidence_score, (int(left*aspect_ratio_x), int(top*aspect_ratio_y-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        cv2.rectangle(base_img, (int(left*aspect_ratio_x), int(top*aspect_ratio_y)), (int(right*aspect_ratio_x), int(bottom*aspect_ratio_y)), (255, 255, 255), 3) #draw rectangle to main image
        
        #-------------------
        #cv2.imshow('face',detected_face)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

In [16]:
cv2.imshow('name', base_img)
cv2.waitKey(0)

113

In [17]:
cv2.destroyAllWindows()